<div style="background:black">
    <center>
<img src="./images/session2/title.png" alt="Title"/>
    </center>
</div>


<div class="alert alert-block alert-success">
<center>
Today's objectives:<br/><br/>
    </center>
    &#x25a2; Get familiar with <b>unsupervised clustering</b><br/>
    &#x25a2; Understand the <b>kmeans</b> algorithm <br/><br/>
    &#x25a2; Practice with Spark <b>RDDs</b><br/>
    &#x25a2; Try <b>Spark's MLlib clustering</b> library <br/>
    &#x25a2; Apply unsupervised clustering to find <b>geographical clusters</b>
</div>


# Unsupervised clustering: the kmeans algorithm

See slides [here](pdf/session2/clustering.pdf).

## Sequential implementation

<div class="alert alert-block alert-info">
<center>
    In this section, we will implement a <b>sequential</b> version of kmeans in Python.<br/>
    In the next section, we will <b>parallelize</b> it in Spark.
</center>
</div>

We will test our implementations on 1,000 simulated 2D points:

In [ ]:
! head 'data/session2/simulated-points.txt'

For convenience, the <code>utils</code> module contains:

(1) A data reader:

In [ ]:
import utils
data_file = 'data/session2/simulated-points.txt'

data_points = utils.read_data(data_file)
data_points[:10]

(2) A data viewer:

In [ ]:
plt = utils.plot_data(data_points, initial_centroids=[], new_centroids=[])
plt.show()

<center>
<div class="alert alert-block alert-info">
<b>Step 1:</b> Initialization <br/>
    </div>
    </center>

<ul style="list-style-image: url('images/do.png');">
<li>Implement function <code>init_centroids</code> as documented in its docstring. Tip: use Python's <code>random.sample</code>. Test your function with k=30.</li>
</ul>

In [ ]:
import random

def init_centroids(data_points, k):
    '''
    Parameters
    ----------
    data_points : data points, as returned by utils.read_data
    k : desired number of centroids

    Returns
    -------
    centroids: a list containing k tuples representing centroids. Example for k=2: [(1,4), (5, 7)]
    '''

In [ ]:
# Test function
k = 30
centroids = init_centroids(data_points, k)
# Check results
centroids[:5]

You can use <code>utls.plot_data</code> to plot your centroids in black over the gray data points:

In [ ]:
# Visualize
plt = utils.plot_data(data_points, initial_centroids=centroids)
plt.show()

<center>
<div class="alert alert-block alert-info">
<b>Step 2:</b> Point assignment to clusters <br/>
    </div>
    </center>

To assign points to clusters, we first need a distance function.

<ul style="list-style-image: url('images/do.png');">
<li>Implement function <code>distance2</code> as documented in the docstring. Tip: the power operator in Python is **. Test your function.
    </ul>

In [ ]:
def distance2(x, y):
    '''
    Parameters
    ----------
    x : a tuple of length 2 representing a data point. Example: (1, 2)
    y : a tuple of length 2 representing a data point. Example: (0, 1)

    Returns
    -------
    distance2: the squared Euclidean distance between x and y
    '''

In [ ]:
distance2((0,1), (1,3))

<ul style="list-style-image: url('images/do.png');">
<li>Implement function <code>closest_class</code> as documented in the docstring. Tip: iterate on centroids using Python's <a href="https://docs.python.org/3/library/functions.html#enumerate"><code>enumerate</code></a>. Test your function.
    </ul>
    <a id="closest_class"/>

In [ ]:
def closest_class(x, centroids):
    '''
    Parameters
    ----------
    x : a tuple of length 2 representing a data point. Example: (1, 2)
    centroids : a list of centroids represented as tuples. Example: [(0,0), (1,3)]

    Returns
    -------
    closest_class: index in centroids of the centroid closest to x
    '''

In [ ]:
closest_class((1,1), [(0,1), (1,1), (2,2)])

<ul style="list-style-image: url('images/do.png');">
<li>Implement function <code>assign_classes</code> as documented in the docstring. Tip: fill list <code>classes</code> with values returned by <code>closest_class</code>.
    </ul>

In [ ]:
def assign_classes(data_points, centroids):
    '''
    Parameters
    ----------
    data_points : data points, as returned by utils.read_data
    centroids : a list of centroids represented as tuples. Example: [(0,0), (1,3)]

    Returns
    -------
    classes: a list such that classes[i] is the index of the centroid in centroids closest to data_points[i]
    '''


<ul style="list-style-image: url('images/do.png');">
<li>The following code plots the result of the first iteration of kmeans. Does it look correct?
    </ul>

In [ ]:
classes = assign_classes(data_points, centroids)
plt = utils.plot_data(data_points, initial_centroids=centroids, classes=classes)
plt.show()

<center>
<div class="alert alert-block alert-info">
<b>Step 3:</b> Centroid update<br/>
    </div>
    </center>

<ul style="list-style-image: url('images/do.png');">
<li>Implement function <code>update_centroids</code> as documented in the docstring.
    </ul>

In [ ]:
def update_centroids(data_points, classes):
    '''
    Parameters
    ----------
    data_points : data points, as returned by utils.read_data
    classes : a list of class indices, returned by assign_classes. Example: [3, 0, 2, 1]

    Returns
    -------
    centroids: a list of tuples such that centroids[i] is the centroid of class i
    '''


<ul style="list-style-image: url('images/do.png');">
<li>The following code plots the updated centroids. Do they look correct?
    </ul>

In [ ]:
new_centroids = update_centroids(data_points, classes)
plt = utils.plot_data(data_points, initial_centroids=centroids, new_centroids=new_centroids, classes=classes)
plt.show()

<center>
<div class="alert alert-block alert-info">
<b>Step 4:</b> Putting it all together<br/>
    </div>
    </center>

<ul style="list-style-image: url('images/do.png');">
<li>We are now ready to implement the main kmeans loop. Fill in the blanks in the skeleton below with the functions implemented previously.
    </ul>

In [ ]:
def kmeans(data_points, k, max_iter=100):
    '''
    Parameters
    ----------
    data_points : data points, as returned by utils.read_data
    k : desired number of centroids
    max_iter : maximum number of iterations after which function will raise an exception
    
    Returns
    -------
    None
    
    Raises
    ------
    exception: if main loop doesn't converge after max_iter iterations
    '''
    
    # Initialization
    initial_centroids = #### BLANK ####
    centroids = initial_centroids
    
    # Main loop
    for i in range(max_iter):
        print(f'kmeans iteration {i}')
        # Point assignment
        classes = #### BLANK ####
        # Centroid update
        new_centroids = #### BLANK ####
        # Convergence test
        if sorted(new_centroids) == sorted(centroids):
            print(f'> Converged')
            return classes, initial_centroids, centroids
        centroids = new_centroids
    raise('x Not converged')

<ul style="list-style-image: url('images/do.png');">
<li>The following code plots the result of our kmeans implementation. Does it look correct?
</ul>
<a id="kmeans-seq"/>

In [ ]:
classes, initial_centroids, final_centroids = kmeans(data_points, k)
plt = utils.plot_data(data_points, initial_centroids=initial_centroids, new_centroids=final_centroids, classes=classes)
plt.show()

## Quiz

<div class="alert alert-block alert-warning">
kmeans is dependent on initialization    

&#x25a2; True

&#x25a2; False
</div>

<div class="alert alert-block alert-info">
kmeans doesn't work with categorical data    

&#x25a2;  True

&#x25a2;  False
</div>

<div class="alert alert-block alert-warning">

The time to run kmeans is proportional to the number of data points
    
&#x25a2; True

&#x25a2; False
</div>

<div class="alert alert-block alert-info">
The point assignment step is the most compute intensive, but it can be parallelized since assignments are done independently for each point.
    
&#x25a2;  True

&#x25a2;  False
</div>

# Parallel implementation in Spark

We will now use Apache Spark to run kmeans in parallel. We will:
1. Implement our own custom version
2. Use the implementation in Spark's machine-learning library

## Custom implementation

Differences with the sequential implementation:


<center>
    <div class="alert alert-block alert-info">
        <b>Data structures to update</b><br/>
    <ul style="list-style: none;">
        <li><code>data_points</code> will now be an RDD</li>
        <li><code>centroids</code> can remain a list</li>
    </ul><br/><br/>
        <b>Functions to re-implement</b><br/>
    <ul style="list-style: none;">
        <li><code>init_centroids</code></li>
        <li><code>assign_classes</code></li>
        <li><code>update_centroids</code></li>
    </ul>
    </div>
    </center>

First, let's create a Spark context:

In [ ]:
# Create a Spark context
from pyspark.context import SparkContext
sc = SparkContext.getOrCreate()

Let's now parse the data in an RDD. Note the use of map functions to:
1. Split the lines by space
2. Convert point coordinates from string to float

In [ ]:
# Parse data
data_points_rdd = sc.textFile('data/session2/simulated-points.txt')\
                    .map(lambda x: x.split(' '))\
                    .map(lambda x: (float(x[0]), float(x[1])))
# Check result
data_points_rdd.take(5)

<center>
<div class="alert alert-block alert-info">
<b>Step 1:</b> Initialization <br/>
    </div>
    </center>

<ul style="list-style-image: url('images/do.png');">
<li>Implement function <code>init_centroids_spark</code> as documented in the docstring. Tip: use Sparks's <a href="https://spark.apache.org/docs/latest/api/python/pyspark.html"><code>takeSample</code></a> transformation. Test your function with k=30.</li>
</ul>

In [ ]:
def init_centroids_spark(data_points_rdd, k):
    '''
    Parameters
    ----------
    data_points_rdd : an RDD containing data points
    k : desired number of centroids

    Returns
    -------
    centroids: a list containing k tuples representing centroids. Example: [(1,4), (5, 7)]
    '''


In [ ]:
# Test function
k = 30
centroids = init_centroids_spark(data_points_rdd, k)
# Check results
centroids[:3]

You can use <code>utls.plot_data</code> to plot your centroids:

In [ ]:
# Visualize
plt = utils.plot_data(data_points_rdd.collect(), initial_centroids=centroids)
plt.show()

<center>
<div class="alert alert-block alert-info">
<b>Step 2:</b> Point assignment to clusters <br/>
    </div>
    </center>

<ul style="list-style-image: url('images/do.png');">
<li>Implement function <code>assign_classes_spark</code> as documented in the docstring. 
    <ul>
        <li><b>Important:</b> make sure that the returned variable stricly complies to the format mentioned in the docstring. If it doesn't, the next steps won't work.</li>
        <li>Tip: use function <a href="#closest_class"><code>closest_class</code></a> from Section 1 and implement this with a single <code>map</code>!
    </ul>

In [ ]:
def assign_classes_spark(data_points_rdd, centroids):
    '''
    Parameters
    ----------
    data_points_rdd : an RDD containing data points
    centroids : a list of centroids represented as tuples. Example: [(0,0), (1,3)]

    Returns
    -------
    classes: an RDD containing tuples (i, p) such that:
       p is a data point in data_points_rdd
       i is the index of the centroid in centroids which is closest to p
       Example: 
       [(23, (-5.97607109113, -2.55353952794)),
        (7, (-5.38734256215, -2.47696382178)),
        (12, (5.78173311542, 6.05513306539))]
    '''


Test your function as follows:

In [ ]:
# Try function
classes_rdd = assign_classes_spark(data_points_rdd, centroids)
# Check result
classes_rdd.take(3)

Visualize the result of the first kmeans iteration:

In [ ]:
# Visualize result
plt = utils.plot_data(data_points_rdd.collect(),
                      initial_centroids=centroids,
                      classes=classes_rdd.map(lambda x: x[0]).collect())
plt.show()

<center>
<div class="alert alert-block alert-info">
<b>Step 3:</b> Centroid update<br/>
    </div>
    </center>

<ul style="list-style-image: url('images/do.png');">
    <li>Implement function <code>update_centroids</code> as documented in the docstring.</li>    
    </ul>

Suggested logic:
<ol>
    <li>map class <code>c</code> to key-value pairs <code>(i, (p, 1))</code>, where <code>i</code> is the class index (<code>c[0]</code>), and <code>p</code> is the data point (<code>c[1]</code>)</li>
    <li>reduce key-value pairs with <code>reduceByKey</code> to obtain pairs <code>(i, ((sum_x, sum_y), n))</code>, where <code>i</code> is a class index, <code>sum_x</code> is the sum of x coordinates of points in class <code>i</code>, <code>sum_y</code> is the sum of y coordinates of points in class <code>i</code>, and <code>n</code> is the number of points in class <code>i</code>.</li>
    <li>map these key-value pairs to <code>(i, centroid)</code>, by dividing <code>sum_x</code> and <code>sum_y</code> by <code>n</code>.</li>
</ol>

In [ ]:
def update_centroids_spark(classes_rdd):
    '''
    Parameters
    ----------
    classes_rdd : an RDD containing tuples (i, p) such that:
       p is a data point in data_points_rdd
       i is the index of the centroid in centroids which is closest to p
       Example: 
       [(23, (-5.97607109113, -2.55353952794)),
        (7, (-5.38734256215, -2.47696382178)),
        (12, (5.78173311542, 6.05513306539))]
        
    Returns
    -------
    centroids: a list of tuples such that centroids[i] is the centroid of class i
    '''


Test your function as follows:

In [ ]:
# Test function
new_centroids = update_centroids_spark(classes_rdd)
# Check result
new_centroids[:3]

Visualize the updated centroids:

In [ ]:
# Visualize
plt = utils.plot_data(data_points_rdd.collect(),
                      initial_centroids=centroids,
                      new_centroids=new_centroids,
                      classes=classes_rdd.map(lambda x: x[0]).collect())
plt.show()

<center>
<div class="alert alert-block alert-info">
<b>Step 4:</b> Putting it all together<br/>
    </div>
    </center>

The final kmeans function is identical to the sequential one, except that it calls the Spark versions of <code>init_centroids</code>, <code>assign_classes</code> and <code>update_centroids</code>:

In [ ]:
def kmeans_spark(data_points_rdd, k, max_iter=100):
    '''
    Parameters
    ----------
    data_points_rdd : data_points_rdd : an RDD containing data points
    k : desired number of centroids
    max_iter : maximum number of iterations after which function will raise an exception
    
    Returns
    -------
    None
    
    Raises
    ------
    exception: if main loop doesn't converge after max_iter iterations
    '''
    
    # Initialization
    initial_centroids = init_centroids_spark(data_points_rdd, k)
    centroids = initial_centroids
    
    # Main loop
    for i in range(max_iter):
        print(f'kmeans iteration {i}')
        # Point assignment
        classes_rdd = assign_classes_spark(data_points_rdd, centroids)
        # Centroid update
        new_centroids = update_centroids_spark(classes_rdd)
        # Convergence test
        if sorted(new_centroids) == sorted(centroids):
            print(f'> Converged')
            return classes_rdd, initial_centroids, centroids
        centroids = new_centroids
    raise('x Not converged')

Test and visualize the result:

In [ ]:
# Test function
classes_rdd, initial_centroids, final_centroids = kmeans_spark(data_points_rdd, k)
# Visualize results
plt = utils.plot_data(data_points_rdd.collect(),
                      initial_centroids=initial_centroids,
                      new_centroids=final_centroids, classes=classes_rdd.map(lambda x: x[0]).collect())
plt.show()

## Spark's Machine-Learning library

<center>
<div class="alert alert-block alert-info">
Spark implements many machine-learning methods, see <a href="http://spark.apache.org/docs/latest/ml-guide.html">here</a>.
    </div>
    </center>

We will use the KMeans implementation from the RDD-based MLLib.

Let's load the data in the format required by the library, an RDD of numeric numpy arrays:

In [ ]:
# Load and parse the data
from numpy import array
data = sc.textFile("data/session2/simulated-points.txt")
parsedData = data.map(lambda line: array([float(x) for x in line.split(' ')]))

# Result sample
parsedData.take(3)

<ul style="list-style-image: url('images/do.png');">
    <li>Use the example <a href="http://spark.apache.org/docs/latest/mllib-clustering.html">here</a> to train a kmeans model called <code>model</code> from <code>parsedData</code>.</li>    
    </ul>

In [ ]:
# Build the model (cluster the data)
from pyspark.mllib.clustering import KMeans, KMeansModel
k = 30
model = KMeans.train(parsedData, k, maxIterations=100, initializationMode="random")

A list of classes suitable for our visualization function can be obtained as follows:

In [ ]:
# Associate classes to data points 
classes = parsedData.map(lambda x: model.predict(x)).collect()

# Result sample
classes[:3]

Visualization:

In [ ]:
# Visualize results

plt = utils.plot_data(parsedData.collect(), new_centroids=model.centers, classes=classes)
plt.show()

## Quiz

<div class="alert alert-block alert-info">
Random centroid selection is the best way to initialize kmeans.
    
&#x25a2; True
    
&#x25a2; False
</div>

<div class="alert alert-block alert-warning">
    To further improve our kmeans implementation, we could parallelize function <code>distance2</code>:
    
&#x25a2; True
    
&#x25a2; False
</div>


# Mini-project: clustering geographical regions

<center>
<div class="alert alert-block alert-info">
    <b>Goal</b>: identify clusters of US states and Canadian provinces/territories based on their flora.
    </div>
    </center>

Data: US Department of Agriculture ([link](https://archive.ics.uci.edu/ml/datasets/Plants)), available in <code>data/session2/plants</code>.

Plausible results:
![Canada](data/session2/results.png)

In your context: plants $\rightarrow$ customer data.

## Data inspection

Main data file:

In [ ]:
! head data/session2/plants/plants.data

State abbreviations:

In [ ]:
! head data/session2/plants/stateabbr.txt

## Helper functions

Implemented in module <code>utils</code>.

(1) Map plots, with colored states:

In [ ]:
import utils
plt = utils.plot_states({'Quebec': 'blue', 'Nova Scotia': 'red', 'Texas': 'yellow', 'California': 'orange'})
plt.show()

<center>
<div class="alert alert-block alert-warning">
    <b>Note:</b> plotting takes a few seconds since the map of Canada, provided by Statistics Canada, is very (too) detailed for our purpose.
    </div>
    </center>

(2) Conversion from state abbreviations (used in the dataset) to state names (used for the plotting):

In [ ]:
utils.convert_state('ca')

(3) Binary encoding of plant lists (returned as a numpy array):

In [ ]:
utils.encode_plant_list(["abelia", "abies", "abies alba"])

## Data preparation for Spark kmeans

Required: an **RDD of arrays containing numbers**.

In our case:
- RDD elements: states
- State arrays: binary encoding of the list of plants found in this state.

Let's first load our data as an RDD:

In [ ]:
filename = 'data/session2/plants/plants.data'

# Create a Spark context
from pyspark.context import SparkContext
sc = SparkContext.getOrCreate()

# Load the text file
plant_rdd = sc.textFile(filename)

# See how the RDD looks like
plant_rdd.take(5)

Let's split the RDD elements by <code>,</code> using a <code>map</code> transformation:

In [ ]:
# Use str.split to split the comma-separated string in each RDD element
plant_rdd = plant_rdd.map(lambda x: x.split(','))

# See how the RDD looks like
plant_rdd.take(2)

<center>
<div class="alert alert-block alert-danger">
    Our RDD still has one element per <b>plant</b>. Instead, we want each RDD element to represent a <b>state</b>.
</div>
</center>

<ul style="list-style-image: url('images/do.png');">
    <li>Transform <code>plant_rdd</code> with a MapReduce job where:
        <ul>
            <li><code>map</code> produces <code>state,plant</code> pairs</li>
            <li><code>reduce</code> aggregates plant names by state</li>
        </ul>
    </li>

<center>
<div class="alert alert-block alert-info">
<b>Note:</b> this is akin to an inverted index
</div>
</center>

<center>
<div class="alert alert-block alert-info">
<code>map</code> produces multiple key-value pairs for each input element $\rightarrow$ use <code>flatMap</code>.
</div>
</center>

<ul style="list-style-image: url('images/do.png');">
    <li>Use a <code>map</code> transformation to convert:
        <ul>
            <li>state abbreviations to names, using <code>utils.convert_state</code></li>
            <li>plant names to binary vectors, using <code>utils.encode_plant_list</code></li>
        </ul>
</ul>

## kmeans classification

Our data is now ready for kmeans classification in Spark. 

<ul style="list-style-image: url('images/do.png');">
    <li>Use Spark's MLlib to classify <code>states_rdd</code> with kmeans. Tip: apply KMeans on an RDD created from <code>states_rdd</code> where the state name is removed</li>
    </ul>

<center>
<div class="alert alert-block alert-warning">
    <b>Note</b>: kmeans classification takes time but our program is ready for deployment on a large-scale cluster!
    </div>
    </center>

Finally, let's associate state names with class numbers:

In [ ]:
classes = states_rdd.map(lambda x: (x[0], clusters.predict(x[1]))).collect()
classes

## Data vizualisation

Let's visualize the result:

In [ ]:
# Map class integers to color names
from matplotlib import colors as mcolors
colors = [ x for x in list(mcolors.XKCD_COLORS.keys()) ]
classes_dict = {}
for c in classes:
    classes_dict[c[0]] = colors[c[1]]

# Plot result
plt = utils.plot_states(classes_dict)
plt.show()

<center>
<div class="alert alert-block alert-info">
    <b>Evaluation:</b> does this result look relevant?
</div>
</center>

# Recap


<div class="alert alert-block alert-success">
    &#x2611; Get familiar with <b>unsupervised clustering</b><br/>
    &#x2611; Understand the <b>kmeans</b> algorithm
</div>
Implemented kmeans in Python

<div class="alert alert-block alert-success">
    &#x2611; Practice with Spark <b>RDDs</b><br/>
</div>
Implemented kmeans in Spark. Transformed geographical data to Spark's MLlib format.

<div class="alert alert-block alert-success">
    &#x2611; Try <b>Spark's MLlib clustering</b> library <br/>
    &#x2611; Apply unsupervised clustering to find <b>geographical clusters</b>
</div>
Clustered US states and Canadian provinces and territories from USDA plant dataset, using Spark's MLlib clustering library.